## Setup

In [ ]:
#! pip install langchain langchain_openai

## Configure OpenAI Settings

In [1]:
# Set up OpenAI
import os
import openai
from dotenv import load_dotenv
# Set up Azure OpenAI
load_dotenv()

OPENAI_API_KEY = os.getenv("PROJECT_OBC_KEY_01").strip()
assert OPENAI_API_KEY, "ERROR: OpenAI Key is missing"

# Option 1: use an OpenAI account
openai_api_key: str = OPENAI_API_KEY

# openai_api_version: str = "2023-05-15"
# model: str = "text-embedding-ada-002"
embedding_model: str = "text-embedding-3-large"

# chat_model: str = "gpt-3.5-turbo-0125"
# MODEL	DESCRIPTION	CONTEXT WINDOW	TRAINING DATA
# gpt-4-0125-preview	New GPT-4 Turbo
# The latest GPT-4 model intended to reduce cases of “laziness” where the model doesn’t complete a task. Returns a maximum of 4,096 output tokens.
# 128,000 tokens	Up to Dec 2023
chat_model: str = "gpt-4-0125-preview"

## Langchain Set up - LLM

In [2]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI

# OpenAI chat model
llm = ChatOpenAI(
    model=chat_model,
    temperature=0, 
    api_key=openai_api_key, 
)

In [3]:
# Test OpenAI chat model
messages = [
    SystemMessage(
        content="You are a helpful assistant to answer user's questions"
    ),
    HumanMessage(
        content="which version of GPT is used? please return 'model_name' as a string."
    ),
]
llm.invoke(messages)
# AIMessage(content="J'adore la programmation.", addiGPT tional_kwargs={}, example=False)

AIMessage(content="'model_name': 'GPT-4'", response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 39, 'total_tokens': 49}, 'model_name': 'gpt-4-0125-preview', 'system_fingerprint': 'fp_f38f4d6482', 'finish_reason': 'stop', 'logprobs': None})

## Prepare FAISS retriever

In [4]:
from langchain_openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

# Use exact the same embedding model
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(
    openai_api_key=openai_api_key, 
    # openai_api_version=openai_api_version, 
    model=embedding_model
)

# Load local FAISS db which we created in earlier using embedding
faiss_db_obc = FAISS.load_local(
    "faiss_index_obc_large",     
    embeddings,
    allow_dangerous_deserialization = True,
)

In [5]:
# List FAISS index
faiss_db_obc.docstore._dict

{'44180098-d8d2-40f4-914a-c74b144109df': Document(page_content=' \n364 6.3.1.1.   Requirem ent for Venting  \n (1)  Except as provided in Art icles 6.3.1.2. and 6.3.1 .3., the products of combustion from solid fuel -burning appliances  \nshall be vented in conformance with the requirements in the applic able appliance  installation standards listed  in Arti cle \n6.2.1.4.  \n6.3.1.2.   Masonry or Concrete Chim neys  \n (1)  Rectangular masonry or concrete chimneys  not more than 12 m in height shall conform to Part 9 if they serve,  \n (a) appliances with a combined total  rated heat output of 120 kW or less, or  \n (b) firepl aces.  \n (2)  Masonry or concrete chimneys  other than those described in Sentence (1) shall be designed and installed in \nconformance with the appropriate requirements in NFPA 211, “Chimneys, Fireplaces, Ven ts and Solid Fuel -Burning \nAppliances”.  \n6.3.1. 3.  Metal Smoke Stacks  \n (1)  Single wall metal smoke s tacks shall be designed and installed in con

In [6]:
# Set up the retriever we want to use, and then turn it into a retriever tool
# We can change the number of results returned with the top_k parameter. The default value is None, which returns all results.
# faiss_retriever_obc = faiss_db_obc.as_retriever(top_k =3)
faiss_retriever_obc = faiss_db_obc.as_retriever()

In [7]:
print(faiss_retriever_obc)

tags=['FAISS', 'OpenAIEmbeddings'] vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000014CFFF59090>


## QnA chain with pipeline-style coding

### Prompt templates

In [ ]:
# ! pip install --upgrade --quiet  langchain langchain-community

In [8]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain import hub


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

#### Optional to create custom prompts for example

In [ ]:
# template = """Answer the question based only on the following context:
# {context}

# Question: {question}
# """
# prompt = ChatPromptTemplate.from_template(template)

In [9]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""
cus_def_prompt = ChatPromptTemplate.from_template(template)

#### Use pre-defined prompt

In [ ]:
from langchain import hub
rag_prompt = hub.pull("rlm/rag-prompt")

In [ ]:
rag_prompt.messages

### Chain without sources

In [10]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

rag_chain = (
    {"context": faiss_retriever_obc | format_docs, 
     "question": RunnablePassthrough()}
    # | rag_prompt
    | cus_def_prompt
    | llm
    | StrOutputParser()
)

In [ ]:
rag_chain.invoke("how big interval shall between two cleanouts on a 6” horizontal sanitary drainage pipe?")

### Chain with sources

In [11]:
from langchain_core.runnables import RunnableParallel

rag_chain_from_docs = (
    # RunnablePassthrough called with assign (RunnablePassthrough.assign(...)) will take the input, and will add the extra arguments passed to the assign function.
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"])))
    # | rag_prompt
    | cus_def_prompt
    | llm
    | StrOutputParser()
)

rag_chain_with_source = RunnableParallel(
    {"context": faiss_retriever_obc, 
     "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)

In [ ]:

rag_chain_with_source.invoke("how big interval shall between two cleanouts on a 6” horizontal sanitary drainage pipe?")

In [16]:
rag_chain_with_source.invoke("how much thickness shall copper sheet be used for vent pipe flashing?")

{'context': [Document(page_content=' \n403  (a) no single chan ge of direction of the vent pipe exceeds 45°,  \n (b) all parts of the vent pipe are nominally vertical , \n (c) the vent pipe is increas ed to not less than 3 in. in size before penetrating a wall or roof, and  \n (d) where the building  is 4 storeys  or less in heig ht, the vent pipe terminates above the roof of the building . \n (4)  Except for a fresh air inlet , where a vent pipe is terminated in  open air , the terminal shall be located,  \n (a) not less than 1 000 mm above or not less than 3.5 m in any other direction fro m every air inlet, openable window or \ndoor,  \n (b) not less than 2 000 mm above or not less than 3.5 m in any other direction from a roof that supports an occupancy , and  \n (c) not less than 2 000 mm above ground.  \n (5)  Where a vent pipe  passes through a roof,  it shall,  \n (a) be terminated high enough to prevent the entry of roof drainage but not less than 150 mm above the roof or above 

In [18]:
rag_chain_with_source.invoke("Information on Vent Pipe Flashing? Explain the answer step by step, and also provide the citation information")

{'context': [Document(page_content=' \n403  (a) no single chan ge of direction of the vent pipe exceeds 45°,  \n (b) all parts of the vent pipe are nominally vertical , \n (c) the vent pipe is increas ed to not less than 3 in. in size before penetrating a wall or roof, and  \n (d) where the building  is 4 storeys  or less in heig ht, the vent pipe terminates above the roof of the building . \n (4)  Except for a fresh air inlet , where a vent pipe is terminated in  open air , the terminal shall be located,  \n (a) not less than 1 000 mm above or not less than 3.5 m in any other direction fro m every air inlet, openable window or \ndoor,  \n (b) not less than 2 000 mm above or not less than 3.5 m in any other direction from a roof that supports an occupancy , and  \n (c) not less than 2 000 mm above ground.  \n (5)  Where a vent pipe  passes through a roof,  it shall,  \n (a) be terminated high enough to prevent the entry of roof drainage but not less than 150 mm above the roof or above 

In [ ]:
# ! pip install langchainhub

In [ ]:
# ! pip install --upgrade --quiet langchain langchain-openai

## Use chain to answer all the questions

#### List of questions: for example:             
plumbing question - cleanouts
question 1:
how big interval shall between two cleanouts on a 4” horizontal sanitary drainage pipe?
section 7.4.7.2 Size and Spacing of Cleanouts (1) (b)
answer: 15m

****************************************************************
question 2:
how big interval shall between two cleanouts on a 6” horizontal sanitary drainage pipe?
section 7.4.7.2 Size and Spacing of Cleanouts (1) (c)
answer: 30m

**************************************************************
question 3:
how many cleanouts shall be placed on a 6” horizontal sanitary drainage pipe with two 90-degree direction changes?
section 7.4.7.1 （5）Cleanouts for Drainage Systems. & section 7.4.7.2 Size and Spacing of Cleanouts (1) (c)
answer: every 30m and at each 90-degree direction change.


#### QnA for OBC

In [ ]:
# # Prompt to make sure the question refers to the right company
# def build_question_prompt(question):
#     prompt_text = f"Please answer the following question: {question}, and explain the answer step by step. \
#         NOTE: if you can find the answer, make sure to provide the source reference as well."
#     return prompt_text

In [12]:
# Prompt to make sure the question refers to the right company
def build_question_prompt(question):
    prompt_text = f"""
    Please answer the following question: {question}
    Explain the answer step by step, and also provide the citation information, 
    for example: 
        Source:
            - document name: "doc name"
            - page number: 123
            - section number: 5.1
    """
    return prompt_text

# Choose a chain who can perform the best for a job given
def agent_in_action(selected_chain, question_prompt):
    result = selected_chain.invoke(question_prompt)    
    return result

def find_answer_in_obc_doc(selected_chain, user_input):    
    question_prompt = build_question_prompt(user_input)
    print(f"Prompt text => {question_prompt}")
    answer = agent_in_action(selected_chain, question_prompt)
    # print(f"Answer: {answer}")
    return answer

In [ ]:
# Unit testing
user_input = "how big interval shall between two cleanouts on a 6” horizontal sanitary drainage pipe?"
# QnA
# agent_on_duty = rag_chain_with_source
agent_on_duty = rag_chain
question_prompt = build_question_prompt(user_input)
print(f"Prompt text => {question_prompt}")
answer = agent_in_action(agent_on_duty, question_prompt)
print(f"Answer: {answer}")

# Chat with OBC Bot

In [13]:
chat_model

'gpt-4-0125-preview'

In [14]:
bot_name = "OBC Bot (" + chat_model + "):"

In [15]:
# TODO: adding chat history ... For now answer question without knowing the chat history
# 
def chat_with_obc_bot(agent_selected):
    while True:
        # 0. Get user input
        user_input = input("You: ")
        print("Me:", user_input)
        if user_input.lower() == 'quit':
            print("Exiting conversation.")
            break  # Exit the loop if user inputs 'quit'

        # bot_response = retriever.get_relevant_documents(user_input)
        bot_response = find_answer_in_obc_doc(agent_selected, user_input)
        # bot_response = summarize_chain.run(docs)

        # print("OBC Bot:", bot_response)
        print(bot_name, bot_response)

# Start the conversation
print("Start chatting with the bot ('quit' to exit):")
# Select one of following chains:
# chain_selected = rag_chain_with_source
chain_selected = rag_chain
chat_with_obc_bot(chain_selected)

Start chatting with the bot ('quit' to exit):
Me: How much thickness shall galvanized steel sheet used for vent pipe flashing?
Prompt text => 
    Please answer the following question: How much thickness shall galvanized steel sheet used for vent pipe flashing?
    Explain the answer step by step, and also provide the citation information, 
    for example: 
        Source:
            - document name: "doc name"
            - page number: 123
            - section number: 5.1
    
OBC Bot (gpt-4-0125-preview): To find the required thickness of galvanized steel sheet used for vent pipe flashing, we need to refer to the section that discusses materials related to vent pipes and their accessories, such as flashing. In the provided context, the relevant information about flashing is found in the section discussing the requirements for when a vent pipe passes through a roof. However, the provided text does not specify the thickness of galvanized steel sheet for vent pipe flashing directly.